In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print('Hello World')

Some references:

Safarov, Rauf. (2020, September). Medical_cost_in_USA, Version 1. Retrieved Sept. 20, 2020 from https://www.kaggle.com/raufsafar/medical-cost-in-usa/notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

plt.style.use('ggplot')

This Dataset has 1338 rows and 7 columns. There is no missing data!

In [ ]:
med_insur = pd.read_csv('../input/insurance/insurance.csv')
med_insur.head(10)

In [ ]:
med_insur.info()

In [ ]:
a = med_insur.isnull().sum()
for index, value in a.items():
    print(f" In column '{index}', there is {value} null objects")

In [ ]:
mean_age = med_insur['age'].mean()
print(f'The avarage of ages is {mean_age}')
mean_bmi = med_insur['bmi'].mean()
print(f'The avarage of BMI is {mean_bmi}')
mean_children = med_insur['children'].mean()
print(f"The avarage number of children is {mean_children}")
mean_charge = med_insur['charges'].mean()
print(f'The avarage of charges is {mean_charge}')

In [ ]:
fig, axs = plt.subplots(2,2,figsize=(15,15))
fig.subplots_adjust(hspace=0.2, wspace=0.2)

ax1 = sns.countplot(x='region', data=med_insur, ax=axs[0][0])
ax2 = sns.countplot(x='children', data=med_insur, ax=axs[0][1])
ax3 = sns.countplot(x='sex', data=med_insur, ax=axs[1][0])
ax4 = sns.countplot(x='smoker', data=med_insur, ax=axs[1][1])

In [ ]:
#How bmi changes with smoking practice??
ax1 = sns.violinplot(x='smoker', y='bmi', hue='sex', data=med_insur)

If you are smoker your bmi is, in average, the same as if you're not a smoker...In our first analysis we can point some differences in the number of people who smoke and people who don't. So I think this can be an important variable for the charge price!

In [ ]:
#Now we can transform our categorical data in dummy variables!
dummy_med_insur = pd.DataFrame()
dummy_med_insur['age'] = med_insur['age']
dummy_med_insur['sex'] = med_insur['sex'].map({'female':0, 'male':1})
dummy_med_insur['bmi'] = med_insur['bmi']
dummy_med_insur['children'] = med_insur['children']
dummy_med_insur['smoker'] = med_insur['smoker'].map({'yes':1, 'no':0})
dummy_med_insur['region'] = med_insur['region'].map({'southeast':0,'southwest':1,'northwest':2,'northeast':3})
dummy_med_insur['charges'] = med_insur['charges']
 
dummy_med_insur.head()

In [ ]:
ax = plt.figure(figsize=(8,5))
ax = sns.heatmap(dummy_med_insur.corr(), annot=True)

We can see a strong correlation between "charges" and "smoker", and a less expressive correlation between "charges" and "age" and "charges" and "bmi".

In [ ]:
ax = sns.pairplot(dummy_med_insur)

The goal of our work is to predict the charge value based on the given parameters. We can see a dispersion in bmi versus charges and age versus charges, but we cleary have a group division in those charts (possible people who smoke and people who don't)

In [ ]:
ax1 = sns.lmplot(x="bmi", y="charges", hue="smoker", data=dummy_med_insur)
ax2 = sns.lmplot(x="age", y="charges", hue="smoker", data=dummy_med_insur)

As we can see, we have two groups with different linear dispersion 'coefficient' in charges versus 'bmi' But in the case of 'charges' vs 'age' we have two subdivisions in the smoker/ no smoker category... but this division is more pronuciate for smokers.

In [ ]:
bmi_sep = dummy_med_insur
bins = [0,30.66,100]
slots = [0,1]
bmi_sep['bmi'] = pd.cut(bmi_sep['bmi'], bins=bins, labels=slots)
bmi_sep.head()

In [ ]:
#Let's take only the smokers!!!

df_mask1=bmi_sep['smoker']==1
filtered_df1 = bmi_sep[df_mask1]
filtered_df1.head()

In [ ]:
ax1 = sns.lmplot(x="age", y="charges", hue="bmi", data=filtered_df1)

Well it seems that if you are a smoker with bmi over 30.66 your charge is around two times more expansive than if you are a smoker with a lower bmi...

In [ ]:
#Let's take only the NO smokers now...!!!

df_mask2=bmi_sep['smoker']==0
filtered_df2 = bmi_sep[df_mask2]
filtered_df2.head()

In [ ]:
ax1 = sns.lmplot(x="age", y="charges", hue="bmi", data=filtered_df2)

For charges to non smokers the bmi seems not to be so important as for people who smoke... Could be interesting make two different models: one for non smokers, descosidering bmi (in first chart, there is no dispersion in bmi), possibly a parabola cold be ideal... and a second, linear model for smokers! But to confirm, let's perform an analysis of variance (ANOVA)...

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

print()
print('Smokers')
print()

mod = ols('charges~bmi+age+bmi*age', data=filtered_df1).fit()
aov = sm.stats.anova_lm(mod, type=2)
print(aov)

print()
print('No Smokers')
print()

mod2 = ols('charges~bmi+age+bmi*age', data=filtered_df2).fit()
aov2 = sm.stats.anova_lm(mod2, type=2)
print(aov2)

As smaller the PR, more important the variable is, as we can see, for non smokers the age is more important than the bmi and for smokers, the bmi is more important than age, but age stills important...

In [ ]:
y_data =filtered_df2['charges'] 
x_data = filtered_df2.drop('charges', axis=1)
x_data.head(100)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.3, random_state=1)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

model = Pipeline([('poly', PolynomialFeatures(degree=2)),
                  ('linear', LinearRegression(fit_intercept=False))])
# fit to an order-2 polynomial data
x = X_train['age']
y = y_train
model = model.fit(x[:, np.newaxis], y)
coefs = model.named_steps['linear'].coef_
print(coefs)

In [ ]:
ax = plt.scatter(x_data['age'],y_data)
m = np.linspace(15,80,1064)
ax2 = plt.plot(m,coefs[2]*m**2+coefs[1]*m+coefs[0], color = 'blue')

In [ ]:
n= np.linspace(15,80,320)
a = coefs[2]*n**2+coefs[1]*n+coefs[0]
b = y_test

In [ ]:
sns.distplot(a-b, kde=False)

The residue distribution are locality around zero, so this seems to be a good model to non smokers. Lets compare with a linear model...

In [ ]:
#Metrics
from sklearn import metrics
print('MAE', metrics.mean_absolute_error(b,a))

In [ ]:
med_insur = pd.read_csv('../input/insurance/insurance.csv')
dummy_med_insur = pd.DataFrame()
dummy_med_insur['age'] = med_insur['age']
dummy_med_insur['sex'] = med_insur['sex'].map({'female':0, 'male':1})
dummy_med_insur['bmi'] = med_insur['bmi']
dummy_med_insur['children'] = med_insur['children']
dummy_med_insur['smoker'] = med_insur['smoker'].map({'yes':1, 'no':0})
dummy_med_insur['region'] = med_insur['region'].map({'southeast':0,'southwest':1,'northwest':2,'northeast':3})
dummy_med_insur['charges'] = med_insur['charges']

In [ ]:
df_mask12=dummy_med_insur['smoker']==1
filtered_df12 = dummy_med_insur[df_mask12]
filtered_df12.head()

In [ ]:
filtered_df12.describe().T

In [ ]:
y_data_nfum = filtered_df12['charges'] 
x_data_nfum = filtered_df12.drop('charges', axis=1)
x_data_nfum.head(100)

X_train, X_test, y_train, y_test = train_test_split(x_data_nfum, y_data_nfum, test_size=0.3, random_state=1)

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
predict = lm.predict(X_test)
plt.scatter(y_test,predict)
print(lm.coef_)

In [ ]:
sns.distplot(y_test-predict, kde=False)

This linear model seems to be good in this case!

last but not least, a overwiew of the contribution of all variables with charges, an Idea of Safarov (see in references). We can see the distribuction of smoker and they are cleary distributed in two groups! Age is the second most important variable for charges. Alone, bmi seems not to be very important, but, as we saw, is that parameter which splits the smoker group...

In [ ]:
df = dummy_med_insur


bins = [17,35,55,1000]
slots = ['Young adult','Senior Adult','Elder']

bins2 = [0,18.5,25,30,100]
slots2 = ['Subpeso', 'Normal', 'Sobrepeso', 'Obeso' ]

df['bmi_range']=pd.cut(dummy_med_insur['bmi'],bins=bins2,labels=slots2)

df['Age_range']=pd.cut(dummy_med_insur['age'],bins=bins,labels=slots)
plt.figure(figsize=(25, 16))
plt.subplot(2,3,1)
sns.violinplot(x = 'smoker', y = 'charges', data = df)
plt.title('Smoker vs Charges',fontweight="bold", size=20)
plt.subplot(2,3,2)
sns.violinplot(x = 'children', y = 'charges', data = df,palette="husl")
plt.title('Children vs Charges',fontweight="bold", size=20)
plt.subplot(2,3,3)
sns.violinplot(x = 'sex', y = 'charges', data = df, palette= 'husl')
plt.title('Sex vs Charges',fontweight="bold", size=20)
plt.subplot(2,3,4)
sns.violinplot(x = 'region', y = 'charges', data = df,palette="bright")
plt.title('Region vs Charges',fontweight="bold", size=20)
plt.subplot(2,3,5)
sns.violinplot(x = 'Age_range', y = 'charges', data = df, palette= 'husl')
plt.title('Age vs Charges',fontweight="bold", size=20)
plt.subplot(2,3,6)
sns.violinplot(x = 'bmi_range', y = 'charges', data = df, palette= 'husl')
plt.title('Bmi vs Charges',fontweight="bold", size=20)



plt.show()

In conclusion, we have analyzed the Medical Insurance data and we propoused two different models to treat smokers and non smokers. That was necessary, because for non smokers, bmi seems not to be vey important as for smokers. This is a little weird, because the avarage of bmi for smokers is the same for non smokers. So, basically, if you are a smoker with a bmi a little bit over 30.7, you will be charged as hell. So unfair... Well, the quadratic regression was ideal for describe the age dependence of charge to non smokers, but as the bmi gain importance to smokers, a linear model coul be adjust and we could see that bmi is anout 6 times more important than age...